In [2]:
# credits:
# https://github.com/nghiaho12/rigid_transform_3D
from numpy import *
from math import sqrt

# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert len(A) == len(B)

    num_rows, num_cols = A.shape;

    if num_rows != 3:
        raise Exception("matrix A is not 3xN, it is {}x{}".format(num_rows, num_cols))

    [num_rows, num_cols] = B.shape;
    if num_rows != 3:
        raise Exception("matrix B is not 3xN, it is {}x{}".format(num_rows, num_cols))

    # find mean column wise
    centroid_A = mean(A, axis=1)
    centroid_B = mean(B, axis=1)
    
    # ensure centroids are 3x1 (necessary when A or B are 
    # numpy arrays instead of numpy matrices)
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - tile(centroid_A, (1, num_cols))
    Bm = B - tile(centroid_B, (1, num_cols))

    H = Am * transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = linalg.svd(H)
    R = Vt.T * U.T

    # special reflection case
    if linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...\n");
        Vt[2,:] *= -1
        R = Vt.T * U.T

    t = -R*centroid_A + centroid_B

    return R, t


In [13]:
set_printoptions(precision=8, suppress=True)

# Число точек
n = 4

# Матрица с точками из которых требуется произвести трансформацию
# Значения по каждой из осей записывается в свою строку
# Здесь средние значения gnss_pose при работе с нулевой трансформацией в launch-файле по осям x, y, z.

A = mat([[ 52.97943878173828,   -59.05085754394531, -52.55366134643555, 60.90013122558594],
         [-58.120079040527344,  -42.9116096496582,  51.418006896972656, 58.78767013549805],
         [ 0.36880460381507874, 0.3688048720359802, 0.36880505084991455, 0.3688013255596161]])

# Матрица с точками в которые требуется произвести трансформацию
# Значения по каждой из осей записывается в свою строку
# Здесь средние значение ndt_pose для каждой из точек по осям x, y, z.

B = mat([[37.35185836708982,    37.352006625430384, 37.35285611146265, 37.35291119241962],
         [-121.95249543930161, -121.9537582843198,  -121.95367313477414, -121.9523913772788],
         [0.36880460381507874,  0.3688048720359802, 0.36880505084991455, 0.3688013255596161]])

import pyproj
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

for j in range (n):
    x, y, z = pyproj.transform(lla, ecef, B[1,j], B[0,j], B[2,j], radians=False)
    print (x, y, z)
    B[0,j] = x
    B[1,j] = y
    B[2,j] = z

# Найти R и t
ret_R, ret_t = rigid_transform_3D(A, B)

# Матрица для сравнения
B2 = (ret_R*A) + tile(ret_t, (1, n))

# Поиск среднеквадратической ошибки
err = B2 - B
err = multiply(err, err)
err = sum(err)
rmse = sqrt(err/n)

print("Points A")
print(A)
print("")

print("Points B")
print(B)
print("")

print("Points B2")
print(B2)
print("")

print("Recovered rotation")
print(ret_R)
print("")

print("Recovered translation")
print(ret_t)
print("")

print("RMSE:", rmse)


-2686510.386752172 -4307257.858082081 3848507.4039813206
-2686600.038174335 -4307190.173857663 3848520.483930775
-2686563.364436543 -4307145.63287559 3848595.428526138
-2686465.0462494055 -4307202.585590661 3848600.2879274637
Points A
[[ 52.97943878 -59.05085754 -52.55366135  60.90013123]
 [-58.12007904 -42.91160965  51.4180069   58.78767014]
 [  0.3688046    0.36880487   0.36880505   0.36880133]]

Points B
[[-2686510.38675217 -2686600.03817433 -2686563.36443654 -2686465.04624941]
 [-4307257.85808208 -4307190.17385766 -4307145.63287559 -4307202.58559066]
 [ 3848507.40398132  3848520.48393077  3848595.42852614  3848600.28792746]]

Points B2
[[-2686510.39392015 -2686600.01894448 -2686563.3559866  -2686465.06676122]
 [-4307257.84077093 -4307190.1764919  -4307145.64859405 -4307202.58454912]
 [ 3848507.4182332   3848520.49436388  3848595.41688352  3848600.2748851 ]]

Recovered rotation
[[ 0.84487019  0.33047596 -0.42068991]
 [-0.53489876  0.50888832 -0.6744746 ]
 [-0.00881346  0.79487     0

In [14]:
# Нахождение кватерниона из матрицы трансформации
from scipy.spatial.transform import Rotation as R
r = R.from_matrix(ret_R)
r.as_quat()

array([ 0.4269854 , -0.11968957, -0.25147427,  0.86030144])